In [2]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as c
import sympy as sy
from sympy import *
from __future__ import print_function, division

In [8]:


def am(werte):
    return np.mean(werte)

def std(werte):
    summen_der_quadrate = 0
    for wert in werte:
        summen_der_quadrate+=(wert-am(werte))**2       
    return np.sqrt(summen_der_quadrate/(werte.size-1))

def stu(werte):
    return std(werte)/np.sqrt(werte.size)

def einfachGaussMult(unsicherheiten, werte, exponenten):
    dz = 0
    for i in range (unsicherheiten.size):
        dz += (exponenten[i]*unsicherheiten[i]/werte[i])**2
    return sqrt(dz)


exponenten = np.array([1,-1])

höhe = 0.4
höhe_stu = 0.005

t_3_mm = np.array([13.13, 13.21, 13.23, 13.30, 13.5, 12.82, 13.18, 12.98, 13.25, 12.77])
t_35_mm = np.array([9.74, 9.81, 9.63, 9.79, 9.73, 9.66, 9.63, 9.75, 9.8, 9.84])
t_4_mm = np.array([7.62, 7.52, 7.50, 7.55, 7.50, 7.48, 7.64, 7.63, 7.47, 7.56])
t_45_mm = np.array([6.09, 6.24, 6.1, 5.96, 6.14, 6.18, 6.21, 6.15, 6.07, 6.01])
t_5_mm = np.array([4.83, 4.99, 5.02, 4.89, 4.97, 5.01, 5.06, 4.95, 4.95, 5.03])
t_55_mm = np.array([4.21, 4.24, 4.27, 4.28, 4.20, 4.04, 4.27, 4.21, 4.21, 4.17])
t_6_mm = np.array([3.62, 3.63, 3.63, 3.72, 3.74, 3.71, 3.72, 3.68, 3.66, 3.54])

t_mm = np.array([t_3_mm, t_35_mm, t_4_mm, t_45_mm, t_5_mm, t_55_mm, t_6_mm])

v = np.zeros(7)
t_stu = np.zeros(7)
deltav_durch_v = np.zeros(7)

for i in range(7):
    t = am(t_mm[i])
    v[i] = höhe/t
    t_stu[i] = stu(t_mm[i])
    
print(v*100, "alles in m/s")

for i in range(7):
    werte = np.array([höhe, am(t_mm[i])])
    unsicherheiten = np.array([höhe_stu,t_stu[i]])
    deltav_durch_v[i] = einfachGaussMult(unsicherheiten, werte, exponenten)
    
print(deltav_durch_v)

deltav = deltav_durch_v * v

print(deltav*100)
    

[ 3.0448352   4.10761963  5.30011925  6.54129191  8.04828974  9.50118765
 10.91405184] alles in m/s
[0.01360208 0.01274028 0.01278248 0.01329093 0.01325113 0.01354926
 0.01357999]
[0.04141609 0.05233223 0.06774867 0.08693985 0.10664896 0.12873409
 0.1482127 ]


In [44]:
def print_to_table(avgs, stds, rows, cols,
                   highlight_best=True,
                   max_is_best=True,
                   avg_format='{0:!>4.1f}',
                   std_format='{1:!<4.1f}',
                   file=None,
                   col_orientation='c'):
    """
    Print data into a LateX table
    The data is given as averages and stadnard deviations.
    It will be printed as 'avgs' \pm 'stds'
    The names of the rows are given in 'rows' and the names of the columns and
    keys to the data dictionaries are given in 'cols'.
    avgs : dict of numpy array with average values
           each numpy array has shape  (N,)
    stds : dict of numpy arrays with standard deviations
           each numpy array shape (N,)
    rows: list of row names, length (N,)
    cols: list of column names and keys to dicts, length (N,)
    highlight_best: best values are printed in bold face, boolean
    max_is_best: whether largest values are best (True) or smallest values, boolean
    avg_format: python string format specifier for printing average values,
                fill-value '!' will be replaced by \phantom{0} in LateX-code,
                since spaces are not respected in LateX code
    std_format: python string format specifier for printing standard deviations,
                see avg_format
    file: file to which the output should be written, string or None
          If file is None, table is printed to stdout
    """
    # Find best values
    best_avg = {}
    if highlight_best:
        if max_is_best:
            for c in cols:
                idx = np.argmax(avgs[c], axis=0)
                best_avg[c] = idx
        else:
            for c in cols:
                idx = np.argmin(avgs[c], axis=0)
                best_avg[c] = idx

    # Header
    print_str = '\\begin{tabular}{l' + (" "+col_orientation) * len(cols) + '} \hline\n'
    for c in cols:
        print_str += ' & ' + c
    print_str += ' \\\\ \n \hline'

    # Content
    for m in range(len(rows)):
        row_name = rows[m]
        print_str += '\n{0}'.format(row_name)
        for c in cols:
            print_str += ' &\n '
            print_list = [avgs[c][m]]
            print_list += [stds[c][m]]
            if m == best_avg.get(c, None):
                print_str += ('$\mathbf{{'+ avg_format + ' \\pm ' + std_format + '}}$').format(*print_list)
            else:
                print_str += ('$'+ avg_format + ' \\pm ' + std_format + '$').format(*print_list)

        print_str += "\n \\vspace*{0.0mm} \\\\"
        print_str = print_str.replace('!', '\\phantom{0}')

    # Footer
    print_str += '\n \hline \n \end{tabular}'

    # Write output
    if file is None:
        print(print_str)
    else:
        with open(file, 'w') as f:
            f.write(print_str)

vdict = {"3mm": v*1000}
deltavdict = {"3mm": deltav*1000}
print_to_table(vdict, deltavdict, ['','','','','','',''], ["3mm"])

\begin{tabular}{l c} \hline
 & 3mm \\ 
 \hline
 &
 $30.4 \pm 0.4\phantom{0}$
 \vspace*{0.0mm} \\
 &
 $41.1 \pm 0.5\phantom{0}$
 \vspace*{0.0mm} \\
 &
 $53.0 \pm 0.7\phantom{0}$
 \vspace*{0.0mm} \\
 &
 $65.4 \pm 0.9\phantom{0}$
 \vspace*{0.0mm} \\
 &
 $80.5 \pm 1.1\phantom{0}$
 \vspace*{0.0mm} \\
 &
 $95.0 \pm 1.3\phantom{0}$
 \vspace*{0.0mm} \\
 &
 $\mathbf{109.1 \pm 1.5\phantom{0}}$
 \vspace*{0.0mm} \\
 \hline 
 \end{tabular}


In [12]:
import pandas as pd
print(pd.DataFrame(v, columns=['v']))

          v
0  0.030448
1  0.041076
2  0.053001
3  0.065413
4  0.080483
5  0.095012
6  0.109141
